<a href="https://colab.research.google.com/github/DiegoLangreo7/EII-RI/blob/main/Ejercicio1_RI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ej. 1. Indexado de una colección estandarizada


En este primer ejercicio, se nos pide indexar una de las dos colecciones de datos propuestas. En mi caso, haremos uso de la base de datos: "TREC-COVID".

Lo primero de todo que haremos, sera preparar nuestro espacio de trabajo para la realizacion de este ejercicio:
Descargamos el paquete "bm25s" requerido para el indexado.

In [ ]:
!pip install bm25s[full]

Una vez instalado, proseguimos con las librerias necesarias.
Estas son:
*   bm25s: Para crear índices y buscar documentos con BM25.
*   Stemmer: Para estematizar términos .
*   json: Para cargar el corpus que nos viene en formato JSON.







In [ ]:
import bm25s
import Stemmer
import json

Con esto tenemos todo lo necesario para comenzar a trabajar.

Ahora descargamos el zip que contiene la colección antes mencionada, con la que trabajaremos:

In [ ]:
!gdown 1SrJV0E1SvYsCt0gkgXmHAOsBVkrAFirs
!unzip -o trec-covid-RI.zip

Downloading...
From (original): https://drive.google.com/uc?id=1SrJV0E1SvYsCt0gkgXmHAOsBVkrAFirs
From (redirected): https://drive.google.com/uc?id=1SrJV0E1SvYsCt0gkgXmHAOsBVkrAFirs&confirm=t&uuid=ade01157-15ef-45f8-9f94-4dcabed5c184
To: /content/trec-covid-RI.zip
100% 71.7M/71.7M [00:00<00:00, 72.6MB/s]
Archive:  trec-covid-RI.zip
  inflating: 3451964.3451965.pdf     
  inflating: corpus.jsonl            
 extracting: LEEME.txt.txt           
  inflating: qrels.tsv               
  inflating: queries.jsonl           


Una vez tenemos la colección, toca parsearla para manejar sus datos:

In [ ]:
corpus_content = []
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        corpus_content.append(json.loads(line.strip()))

Ya tenemos el archivo abierto y leido, pero ahora necesitamos preparar nuestro corpus para BM25S en dos formatos:


1.   Una versión literal que mantenga la estructura original de los "documentos".
2.   Una versión de texto plano para tokenización e indexación.





In [ ]:
corpus_verbatim = list()
corpus_plaintext = list()
for entry in corpus_content:
  value = value+1
  document = {"id": entry["_id"], "title": entry["title"].lower(),
  "text": entry["text"].lower()}
  corpus_verbatim.append(document)
  corpus_plaintext.append(f"{entry['title'].lower()} {entry['text'].lower()}")

171332


Ahora toca procesar el texto. Como podemos comprobar al abrir el archivo .jsonl en el bloc de notas o notepad++, es que el idioma del mismo es el inglés, por tanto, aplicamos un stemmer para el idioma inglés y eliminamos palabras
vacías al realizar la tokenización:

In [ ]:
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en",
 stemmer=stemmer, show_progress=True)


Split strings:   0%|          | 0/171332 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

Ahora crearemos nuestro retriever basado en BM25 e indexaremos el corpus tokenizado.

In [ ]:
bm25_flavor = "lucene"
idf_flavor = "lucene"

retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True)


DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/171332 [00:00<?, ?it/s]

Una vez ejecutado este ultimo fragmento, la coleccion ya estara indexada. Lo que podemos hacer es tambien almacenarla:

In [ ]:
retriever.save("COVID", corpus=corpus_verbatim)

Finding newlines for mmindex:   0%|          | 0.00/199M [00:00<?, ?B/s]